<a href="https://colab.research.google.com/github/z-tufekci/DeepLearning/blob/main/GenerateBlogPostswithGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install transformers

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

In [ ]:
tokenizer.decode(tokenizer.eos_token_id)

'<|endoftext|>'

In [ ]:
sentence="I don't like chocolate but I can consume it with coffe"
input_ids = tokenizer.encode(sentence, return_tensors='pt')

In [ ]:
input_ids # change it to number decoders

tensor([[   40,   836,   470,   588, 11311,   475,   314,   460, 15000,   340,
           351,   763, 16658]])

In [ ]:
# Beam search 
output = model.generate(input_ids, max_length=200, num_beams=5, no_repeat_ngram_size=2, early_stopping=True )


In [ ]:
text = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
with open('blogpost.text', 'w') as f:
    f.write(text)

# **PART 2**   
# ⛳ AI Blog Post Summarization with Hugging Face Transformers
# & Beautiful Soup Web Scraping  ⛳


In [24]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [25]:
summerizer =pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [53]:
# https://medium.com/augmented-startups/is-pp-yoloe-better-than-yolov5-yolox-988386969bcf
URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"
r = requests.get(URL)

In [ ]:
#r.text

In [57]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['p', 'h1'])

In [58]:
results

[<h1>Will The Game Stop with Gamestop Or Is This Just The Beginning?</h1>,
 <p>Crypto, Markets, Trading</p>,
 <p class="paragraph">The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business.</p>,
 <p class="paragraph">The events can be interpreted with many viable lenses and there are ironies baked in that are pure serendipity. There has been a centrally controlled game in the global financial system in which insiders benefited while outsiders got hurt that comes to a head with a company called GameStop. The broking firm of most of the retail side of this warfare ‘RobinHood’ is literally stealing from its poor, retail investors to give to its rich, capital backers.</p>,
 <p class="paragraph">One of the historical realities of this game has been that macro-investing – the sages of not only portfolio management, but ofte

In [74]:
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [ ]:
#ARTICLE

In [76]:
ARTICLE = ARTICLE.replace('. ','.<eos>')
ARTICLE = ARTICLE.replace('! ','!<eos>')
ARTICLE = ARTICLE.replace('? ','?<eos>')
sentence = ARTICLE.split('<eos>')

In [77]:
sentence

['Will The Game Stop with Gamestop Or Is This Just The Beginning?',
 'Crypto, Markets, Trading The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business.',
 'The events can be interpreted with many viable lenses and there are ironies baked in that are pure serendipity.',
 'There has been a centrally controlled game in the global financial system in which insiders benefited while outsiders got hurt that comes to a head with a company called GameStop.',
 'The broking firm of most of the retail side of this warfare ‘RobinHood’ is literally stealing from its poor, retail investors to give to its rich, capital backers.',
 'One of the historical realities of this game has been that macro-investing – the sages of not only portfolio management, but often also sophisticated social and cultural figures – have had a hard time ma

In [81]:
max_chunk = 500
current_chunk = 0
chunks = []

for sen in sentence:
    if len(chunks) == current_chunk +1:
        if len(chunks[current_chunk]) + len(sen.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sen.split(' '))
        else:
            current_chunk += 1
            chunks.append(sen.split(' '))
    else:
        #print(current_chunk)
        chunks.append(sen.split(' '))


In [82]:
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id]) 

In [83]:
chunks[0]

'Will The Game Stop with Gamestop Or Is This Just The Beginning? Crypto, Markets, Trading The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business. The events can be interpreted with many viable lenses and there are ironies baked in that are pure serendipity. There has been a centrally controlled game in the global financial system in which insiders benefited while outsiders got hurt that comes to a head with a company called GameStop. The broking firm of most of the retail side of this warfare ‘RobinHood’ is literally stealing from its poor, retail investors to give to its rich, capital backers. One of the historical realities of this game has been that macro-investing – the sages of not only portfolio management, but often also sophisticated social and cultural figures – have had a hard time making money in markets

In [85]:
len(chunks[1].split(' '))

473

In [88]:
res = summerizer(chunks, max_length=120, min_length=30, do_sample=False)

In [89]:
res

[{'summary_text': ' The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business . The broking firm of most of the retail side of this warfare ‘RobinHood’ is stealing from its poor, retail investors to give to its rich, capital backers .'},
 {'summary_text': ' Victor Niederhoffer blew his hedge fund up in 1997 in a highly statistically improbable event, in which he sold puts that were targeted by market mechanics, rather than ‘truth’. The market here is more leveraged, more volatile, more aggressive, better for types of trading and worse for investing. The fundamentals no longer matter; and this was demonstrated only 5 years later with what could be the largest bubble of viable assets in history .'},
 {'summary_text': ' Greenspans moral hazard of late 1994/1995 fundamentally changed the market in a new way . Extraordinar

In [90]:
len(res)

7

In [93]:
text = ' '.join( [summ ['summary_text']  for summ in res] )

In [94]:
with open('summary.txt', 'w') as f:
    f.write(text)


# AI Blog Post Summarization with Hugging Face Transformers & Beautiful Soup Web Scraping by Nicholas Renotte 
# https://www.youtube.com/watch?v=JctmnczWg0U 
